# Required library

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init(max_mem_size='16G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwbd06uwo
  JVM stdout: /tmp/tmpwbd06uwo/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpwbd06uwo/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_221j18
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


# Road and check the data

In [3]:
# Read the data
path = '/kaggle/input/stanford-covid-vaccine'
train_df = pd.read_json(f'{path}/train.json',lines=True)
test_df = pd.read_json(f'{path}/test.json', lines=True)
sample_sub_df = pd.read_csv(f'{path}/sample_submission.csv')

In [4]:
train_df.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [5]:
test_df.head()

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [6]:
sample_sub_df.head()

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0


# Definition of objective and explanatory variables
- Quote from
- https://www.kaggle.com/kaushal2896/openvaccine-xgboost-baseline

In [7]:
# Calculate Means of targets
train_df['reactivity'] = train_df['reactivity'].apply(lambda x: np.mean(x))
train_df['deg_Mg_pH10'] = train_df['deg_Mg_pH10'].apply(lambda x: np.mean(x))
train_df['deg_pH10'] = train_df['deg_pH10'].apply(lambda x: np.mean(x))
train_df['deg_Mg_50C'] = train_df['deg_Mg_50C'].apply(lambda x: np.mean(x))
train_df['deg_50C'] = train_df['deg_50C'].apply(lambda x: np.mean(x))

In [8]:
train_df.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...",0.502631,0.559628,0.503501,0.538540,0.585893
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...",0.411401,0.698354,0.846041,0.930103,0.865125
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...",0.433966,0.578362,0.548174,0.570284,0.575172
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...",0.329059,0.826485,1.051551,1.221297,0.696476
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...",0.282204,0.469587,0.384826,0.393072,0.440115


In [9]:
# Drop unnecessary columns for now
train_df = train_df.drop(['id', 'index', 'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C', 'SN_filter', 'signal_to_noise', 'deg_pH10', 'deg_50C'], axis=1)
train_df.head()

,sequence,structure,predicted_loop_type,seq_length,seq_scored,reactivity,deg_Mg_pH10,deg_Mg_50C
0,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,107,68,0.502631,0.559628,0.538540
1,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,107,68,0.411401,0.698354,0.930103
2,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,107,68,0.433966,0.578362,0.570284
3,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,107,68,0.329059,0.826485,1.221297
4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,107,68,0.282204,0.469587,0.393072


In [10]:
# Split data in features and labels
X_train = train_df.drop(['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'], axis=1)
Y_train = train_df[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.15)

In [12]:
def featurize(df):
    
    df['total_A_count'] = df['sequence'].apply(lambda s: s.count('A'))
    df['total_G_count'] = df['sequence'].apply(lambda s: s.count('G'))
    df['total_U_count'] = df['sequence'].apply(lambda s: s.count('U'))
    df['total_C_count'] = df['sequence'].apply(lambda s: s.count('C'))
    
    df['total_dot_count'] = df['structure'].apply(lambda s: s.count('.'))
    df['total_ob_count'] = df['structure'].apply(lambda s: s.count('('))
    df['total_cb_count'] = df['structure'].apply(lambda s: s.count(')'))
    
    return df

In [13]:
X_train = featurize(X_train)
X_test = featurize(X_test)

In [14]:
X_train = X_train.drop(['sequence', 'structure', 'predicted_loop_type'], axis=1)
X_test = X_test.drop(['sequence', 'structure', 'predicted_loop_type'], axis=1)

In [15]:
X_train.head()

,seq_length,seq_scored,total_A_count,total_G_count,total_U_count,total_C_count,total_dot_count,total_ob_count,total_cb_count
1189,107,68,69,7,25,6,93,7,7
1183,107,68,24,36,17,30,59,24,24
2274,107,68,43,24,20,20,59,24,24
915,107,68,38,24,17,28,51,28,28
2069,107,68,45,30,16,16,61,23,23


In [16]:
X_test.head()

,seq_length,seq_scored,total_A_count,total_G_count,total_U_count,total_C_count,total_dot_count,total_ob_count,total_cb_count
367,107,68,43,23,19,22,55,26,26
1942,107,68,76,16,8,7,93,7,7
1041,107,68,39,38,3,27,71,18,18
2104,107,68,31,34,12,30,49,29,29
1045,107,68,44,24,16,23,59,24,24


In [17]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# AutoML - train & predict
- Reference:
- https://www.kaggle.com/tunguz/ieee-with-h2o-automl
- https://stackoverflow.com/questions/59366199/is-there-a-way-to-convert-h2oframe-to-pandas-dataframe
- https://stackoverflow.com/questions/48317921/python-h2o-frame-to-np-array-reshape

In [18]:
htrain = pd.concat([Y_train, pd.DataFrame(X_train)], axis=1)
htrain = h2o.H2OFrame(htrain)
htrain['reactivity'].asfactor
htrain['deg_Mg_pH10'].asfactor
htrain['deg_Mg_50C'].asfactor
htrain

Parse progress: |█████████████████████████████████████████████████████████| 100%


reactivity,deg_Mg_pH10,deg_Mg_50C,0,1,2,3,4,5,6,7,8
0.502631,0.559628,0.53854,0,0,2.5612,-2.39057,0.935213,-2.38768,2.11724,-2.11724,-2.11724
nan,nan,nan,0,0,-1.7442,1.49978,-0.0154956,1.01941,-0.0330479,0.0330479,0.0330479
0.433966,0.578362,0.570284,0,0,0.0736327,-0.110016,0.34102,-0.400208,-0.0330479,0.0330479,0.0330479
nan,nan,nan,0,0,-0.404745,-0.110016,-0.0154956,0.73549,-0.538997,0.538997,0.538997
0.282204,0.469587,0.393072,0,0,0.264984,0.694884,-0.134334,-0.968057,0.0934394,-0.0934394,-0.0934394
nan,nan,nan,0,0,-0.30907,0.158284,-0.372011,0.73549,-0.538997,0.538997,0.538997
0.349713,0.353104,0.367032,0,0,-1.17015,0.829034,0.578697,0.167641,-0.538997,0.538997,0.538997
0.304882,0.443834,0.336829,0,0,0.456335,-0.914917,-0.372011,0.73549,-0.0330479,0.0330479,0.0330479
0.220175,0.310119,0.313763,0,0,0.36066,0.560734,-0.372011,-0.684133,0.219927,-0.219927,-0.219927
0.436157,0.49866,0.381422,0,0,2.65687,-2.25642,-1.20388,-0.116284,2.11724,-2.11724,-2.11724


In [19]:
htest = pd.concat([Y_test, pd.DataFrame(X_test)], axis=1)
htest = h2o.H2OFrame(htest)
htest['reactivity'].asfactor
htest['deg_Mg_pH10'].asfactor
htest['deg_Mg_50C'].asfactor
htest

Parse progress: |█████████████████████████████████████████████████████████| 100%


reactivity,deg_Mg_pH10,deg_Mg_50C,0,1,2,3,4,5,6,7,8
nan,nan,nan,0,0,0.0736327,-0.244166,0.222181,-0.116284,-0.286022,0.286022,0.286022
0.411401,0.698354,0.930103,0,0,3.23093,-1.18322,-1.08504,-2.24572,2.11724,-2.11724,-2.11724
nan,nan,nan,0,0,-0.30907,1.76808,-1.67924,0.593527,0.725876,-0.725876,-0.725876
0.329059,0.826485,1.2213,0,0,-1.07447,1.23148,-0.609688,1.01941,-0.665484,0.665484,0.665484
nan,nan,nan,0,0,0.169308,-0.110016,-0.134334,0.0256785,-0.0330479,0.0330479,0.0330479
0.286488,0.411699,0.325238,0,0,1.22174,-0.914917,-0.253173,-0.54217,-0.0330479,0.0330479,0.0330479
nan,nan,nan,0,0,2.65687,-0.780767,-0.728527,-2.24572,2.11724,-2.11724,-2.11724
nan,nan,nan,0,0,-0.691772,0.158284,-0.0154956,0.877452,-1.42441,1.42441,1.42441
nan,nan,nan,0,0,-0.787448,0.829034,-0.609688,1.01941,-0.665484,0.665484,0.665484
nan,nan,nan,0,0,-0.213394,0.158284,0.578697,-0.54217,-0.538997,0.538997,0.538997


In [20]:
# Convert test data to predict
test = featurize(test_df.drop(['index', 'id'], axis=1))
test = test.drop(['sequence', 'structure', 'predicted_loop_type'], axis=1)
test = scaler.transform(test)
test = h2o.H2OFrame(test, column_names=['0','1','2','3','4','5','6','7','8'])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
# train & predict an each
def h2o_ML(htrain, htest, test):
    Y1 = 'reactivity'
    Y2 = 'deg_Mg_pH10'
    Y3 = 'deg_Mg_50C'
    X1 = htrain.columns.remove(Y1)
    X2 = htrain.columns.remove(Y2)
    X3 = htrain.columns.remove(Y3)
    
    def h2o_train_test(X, Y, htrain, htest, test):       
        aml = H2OAutoML(max_runtime_secs=(3600 * 2),  # 2 hours for all Y
                        max_models=30, # None, # no limt
                        seed=2000,
                        nfolds=3,
                        keep_cross_validation_predictions=True
                       )
        aml.train(x=X, y=Y, training_frame=htrain, leaderboard_frame=htest)
        
        # View the AutoML Leaderboard
        lb = aml.leaderboard
        print(lb.head(rows=lb.nrows))  # Print all rows instead of default (10 rows)
        
        # predict
        preds = aml.predict(test).as_data_frame().values
        preds = pd.DataFrame(preds, columns=[Y])
        
        return preds
        
    h2o_1 = h2o_train_test(X1, Y1, htrain, htest, test)
    h2o_2 = h2o_train_test(X2, Y2, htrain, htest, test)
    h2o_3 = h2o_train_test(X3, Y3, htrain, htest, test)
    
    result = pd.concat([h2o_1, h2o_2, h2o_3], axis=1)
    
    return result

In [22]:
# Run functions
model = h2o_ML(htrain, htest, test)
model

AutoML progress: |
22:54:45.344: XGBoost_1_AutoML_20200920_225444 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_20200920_225444_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█
22:54:46.349: XGBoost_2_AutoML_20200920_225444 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_20200920_225444_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.



22:54:47.354: XGBoost_3_AutoML_20200920_225444 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_20200920_225444_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


████████████████
22:5

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid__1_AutoML_20200920_225444_model_6,0.00738631,0.0859436,0.00738631,0.0545764,0.0628694
GBM_3_AutoML_20200920_225444,0.0074295,0.0861945,0.0074295,0.0553048,0.0628458
GBM_grid__1_AutoML_20200920_225444_model_7,0.0074473,0.0862977,0.0074473,0.0544819,0.0626223
GBM_2_AutoML_20200920_225444,0.00750497,0.0866312,0.00750497,0.0552353,0.0629706
GBM_5_AutoML_20200920_225444,0.00752197,0.0867293,0.00752197,0.0546333,0.0636622
GBM_grid__1_AutoML_20200920_225444_model_1,0.00755649,0.086928,0.00755649,0.0544746,0.0632809
GBM_grid__1_AutoML_20200920_225444_model_10,0.00756139,0.0869563,0.00756139,0.0589072,0.0619725
GBM_grid__1_AutoML_20200920_225444_model_5,0.00759711,0.0871614,0.00759711,0.0562014,0.0641314
DeepLearning_grid__3_AutoML_20200920_225444_model_2,0.0076264,0.0873292,0.0076264,0.0553775,0.0646547
GBM_grid__1_AutoML_20200920_225444_model_8,0.00764455,0.0874331,0.00764455,0.0550484,0.0637525



gbm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'deg_Mg_pH10': substituting in a column of NaN
  warnings.warn(w)
/opt/conda/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'deg_Mg_50C': substituting in a column of NaN
  warnings.warn(w)


AutoML progress: |
23:20:40.821: XGBoost_1_AutoML_20200920_232039 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_20200920_232039_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█
23:20:41.823: XGBoost_2_AutoML_20200920_232039 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_20200920_232039_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█
23:20:42.825: XGBoost_3_AutoML_20200920_232039 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_20200920_232039_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


██████████████
23:20

...

/opt/conda/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'reactivity': substituting in a column of NaN
  warnings.warn(w)


AutoML progress: |
23:48:25.202: XGBoost_1_AutoML_20200920_234824 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_20200920_234824_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█
23:48:26.205: XGBoost_2_AutoML_20200920_234824 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_20200920_234824_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█
23:48:27.207: XGBoost_3_AutoML_20200920_234824 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_20200920_234824_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


█████████████
23:48:

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200920_234824,0.010006,0.10003,0.010006,0.06154,0.078252
GBM_1_AutoML_20200920_234824,0.0101166,0.100581,0.0101166,0.0613664,0.0770673
StackedEnsemble_BestOfFamily_AutoML_20200920_234824,0.0101371,0.100683,0.0101371,0.0631146,0.0781736
DeepLearning_grid__3_AutoML_20200920_234824_model_3,0.0103028,0.101502,0.0103028,0.066374,0.0778297
GBM_3_AutoML_20200920_234824,0.0105303,0.102617,0.0105303,0.0630167,0.0793493
DeepLearning_grid__3_AutoML_20200920_234824_model_1,0.010566,0.102791,0.010566,0.0647867,0.0827198
GBM_grid__1_AutoML_20200920_234824_model_7,0.0106444,0.103172,0.0106444,0.0617179,0.079731
GBM_grid__1_AutoML_20200920_234824_model_3,0.0106946,0.103415,0.0106946,0.0614676,0.0795407
DeepLearning_grid__2_AutoML_20200920_234824_model_2,0.0107527,0.103695,0.0107527,0.0685172,0.0765239
GBM_2_AutoML_20200920_234824,0.0107732,0.103794,0.0107732,0.062418,0.0795746



stackedensemble prediction progress: |████████████████████████████████████| 100%


,reactivity,deg_Mg_pH10,deg_Mg_50C
0,0.396706,0.454717,0.408552
1,0.400306,0.454101,0.402659
2,0.397701,0.451536,0.410363
3,0.399495,0.450908,0.405003
4,0.417669,0.444551,0.416413
...,...,...,...
3629,0.415873,0.457682,0.405266
3630,0.395881,0.456338,0.402828
3631,0.410651,0.458140,0.415940
3632,0.395894,0.452541,0.409513


In [23]:
h2o.shutdown(prompt=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  """Entry point for launching an IPython kernel.


H2O session _sid_8377 closed.


# Submit
- Reference:
- https://www.kaggle.com/kaushal2896/openvaccine-xgboost-baseline

In [24]:
# Create submission csv
submission_df = model.loc[model.index.repeat(list(test_df['seq_length']))].reset_index(drop=True)
submission_df = submission_df.rename(columns={0: 'reactivity', 1: 'deg_Mg_pH10', 2: 'deg_Mg_50C'})
submission_df['id_seqpos'] = sample_sub_df['id_seqpos']
submission_df['deg_pH10'] = 0.0
submission_df['deg_50C'] = 0.0
submission_df = submission_df[['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
submission_df

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.396706,0.454717,0.0,0.408552,0.0
1,id_00073f8be_1,0.396706,0.454717,0.0,0.408552,0.0
2,id_00073f8be_2,0.396706,0.454717,0.0,0.408552,0.0
3,id_00073f8be_3,0.396706,0.454717,0.0,0.408552,0.0
4,id_00073f8be_4,0.396706,0.454717,0.0,0.408552,0.0
...,...,...,...,...,...,...
457948,id_ffda94f24_125,0.399154,0.458056,0.0,0.405797,0.0
457949,id_ffda94f24_126,0.399154,0.458056,0.0,0.405797,0.0
457950,id_ffda94f24_127,0.399154,0.458056,0.0,0.405797,0.0
457951,id_ffda94f24_128,0.399154,0.458056,0.0,0.405797,0.0


In [25]:
# Save that CSV for submission
submission_df.to_csv('submission.csv', index=False)